# Mental health counseling **Clam Mind Bot**

## Install Packages

In [ ]:

!pip install ipykernel
!python.exe -m pip install --upgrade pip
!pip install openai datasets


## Task 1: Import the Libraries

In [20]:
# Import the libraries here
import os as os
from openai import OpenAI
from datasets import load_dataset
from enum import Enum
import random as random
import json as json

## Task 2: Create Classes for the Dataset

In [21]:
#  Define the RoleType Enum here
class RoleType(Enum):
    USER = 'user'
    SYSTEM = 'system'
    ASSISTANT = 'assistant'

In [22]:
# Define the Role class here
class Role(object):
    def __init__(self, role_type:RoleType, content):
        self.role = role_type.value
        self.content = content
        self.value = {'role': self.role, 'content':self.content } 

In [26]:
#  Define the messsage class here
class Message(object):
    def __init__(self, user_content, system_content, assistant_content):
        self.user_role = Role(role_type=RoleType.USER , content=user_content)
        self.system_role = Role(role_type=RoleType.SYSTEM , content=system_content)
        self.assistant_role = Role(role_type=RoleType.ASSISTANT, content=assistant_content)
        self.message = {'messages':[self.system_role.value, self.user_role.value, self.assistant_role.value, ]} 

## Task 3: Load, Explore, and Store the Data

In [ ]:
# Load the dataset 
dataset = load_dataset("Amod/mental_health_counseling_conversations", split = 'train')

In [ ]:
# Create a sample Message object
context = dataset[152]['Context']
response = dataset[152]['Response']
system_content = "You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user’s emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It’s important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsibility is to create a secure space for users to express themselves and reflect. Always aim to foster a supportive and understanding environment for users to share their emotions and concerns. Above all, prioritize their well-being and safety."
message_obj = Message(user_content=context, system_content=system_content, assistant_content = response )

print(message_obj.message)

In [ ]:
#  Create the train_dataset variable
# Sample 100 items from the 'train' split
sampled_dataset = random.choices(dataset, k=100)
train_dataset = []

# Print the sampled data to verify
print(sampled_dataset[1])

for row in sampled_dataset:
    message_obj = Message(user_content=row['Context'], system_content=system_content, assistant_content=row['Response'])
    train_dataset.append(message_obj.message)

print(train_dataset[1])

In [31]:
# Save data in JSONl format 
def save_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for row in data:
            line = json.dumps(row)
            file.write(line + '\n')


# Store the data in JSONL format
training_data_path = '../data/train.jsonl'
save_to_jsonl(train_dataset[:-5], training_data_path)

validation_data_path = '../data/validation.jsonl'
save_to_jsonl(train_dataset[-5:], validation_data_path)

## Task 4: Fine-Tune the Model

In [32]:
# Load the training and validation files
training_data = open(training_data_path, "rb")
validation_data = open(validation_data_path, "rb")

In [33]:
#  Add OpenAI api_key
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [34]:
# Upload the training and validation files
training_response = client.files.create(file=training_data, purpose="fine-tune")
training_file_id = training_response.id

validation_response = client.files.create(file=validation_data, purpose="fine-tune")
validation_file_id = validation_response.id

print("Training file id:", training_file_id)
print("Validation file id:", validation_file_id)

Training file id: file-JsjCy6KBqzUkHthW7xx3QW
Validation file id: file-GoSXp6zn3mjdjqwUcBTaaL


In [35]:
# Create a fine-tuning job
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo",
    suffix="clammindbot-test-model",
    validation_file=validation_file_id
)

job_id = response.id

print(response)

FineTuningJob(id='ftjob-dqylTcKD9xMDgXlzFfT35L9g', created_at=1744643331, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-Wkgb4Ms6IvxLjiylKBEA4atV', result_files=[], seed=1673516955, status='validating_files', trained_tokens=None, training_file='file-JsjCy6KBqzUkHthW7xx3QW', validation_file='file-GoSXp6zn3mjdjqwUcBTaaL', estimated_finish=None, integrations=[], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix='clammindbot-test-model')


In [37]:
# Retrieve the job status
job_id = response.id
print("Job ID:", job_id)

job_status = client.fine_tuning.jobs.retrieve(job_id)
print(job_status)

Job ID: ftjob-dqylTcKD9xMDgXlzFfT35L9g
FineTuningJob(id='ftjob-dqylTcKD9xMDgXlzFfT35L9g', created_at=1744643331, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-Wkgb4Ms6IvxLjiylKBEA4atV', result_files=[], seed=1673516955, status='validating_files', trained_tokens=None, training_file='file-JsjCy6KBqzUkHthW7xx3QW', validation_file='file-GoSXp6zn3mjdjqwUcBTaaL', estimated_finish=None, integrations=[], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3)), type='supervised'), user_provided_suffix='clammindbot-test-model')


## Task 5: Test the Fine-Tuned Model

In [38]:
# Create and store message dictionaries
system_message = """You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user's emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It's important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsibility is to create a secure space for users to express themselves and reflect. Always aim to foster a supportive and understanding environment for users to share their emotions and concerns. Above all, prioritize their well-being and safety."""

messages = []
messages.append({"role": "system", "content": system_message})
user_message = "Every winter I find myself getting sad because of the weather. How can I fight this?"
messages.append({"role": "user", "content": user_message})

In [40]:
# Test the fine-tuned chat completion model
completion = client.chat.completions.create(
    model= response.model,
    messages=messages
)
print(completion.choices[0].message)

ChatCompletionMessage(content="I hear you, and it's understandable to feel down during the winter months. You might be experiencing Seasonal Affective Disorder (SAD), a type of depression that's related to changes in seasons. Here are some strategies that might help you combat these feelings:\n\n1. **Light Therapy**: Consider using a light therapy box, which mimics natural sunlight and can help improve your mood.\n\n2. **Maintain a Routine**: Try to stick to a regular schedule, including sleep, exercise, and mealtimes. This can help stabilize your mood.\n\n3. **Stay Active**: Engaging in regular physical activity, even if it's just a short walk outside, can boost your mood and energy levels.\n\n4. **Connect with Others**: Stay socially connected with friends and loved ones, whether in person or virtually. Having a support system can make a big difference.\n\n5. **Practice Self-Care**: Take time for activities you enjoy, practice relaxation techniques, and prioritize self-care.\n\n6. **

In [41]:
# Get and compare the output of the gpt-3.5-turbo chat completion model
completion = client.chat.completions.create(
    model= "gpt-3.5-turbo",
    messages=messages
)
print(completion.choices[0].message)

ChatCompletionMessage(content="It's common to feel down during the winter months due to the lack of sunlight and colder weather. One way to combat these feelings is by practicing self-care and seeking out activities that boost your mood. Here are a few tips that might help:\n\n1. **Get sunlight**: Try to spend time outdoors during daylight hours or sit near a window to get natural light, which can improve your mood.\n\n2. **Stay active**: Engaging in regular physical activity, even brief walks or at-home workouts, can help reduce feelings of sadness.\n\n3. **Maintain a routine**: Stick to a regular schedule for sleeping, eating, and activities to provide a sense of structure and stability.\n\n4. **Connect with others**: Stay connected with friends and family, even if it's through virtual means. Social support can be very beneficial.\n\n5. **Practice relaxation techniques**: Activities like deep breathing, meditation, or yoga can help reduce stress and promote relaxation.\n\n6. **Consid